In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, GRU, TimeDistributed, BatchNormalization
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
input_shape = (128, 128, 3)  # Resize images to 128x128
batch_size = 64
num_classes = 26

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

valid_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    "Augmented_data/train",
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode="categorical",
)

valid_generator = valid_datagen.flow_from_directory(
    "Augmented_data/valid",
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode="categorical",
)

# CNN Model for Image Classification
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation="relu", input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(num_classes, activation="softmax"),
    ])
    return model

model = build_cnn_model(input_shape, num_classes)

# Compile model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=valid_generator.samples // batch_size,
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Conv1D, Reshape, BatchNormalization, Activation
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
input_shape = (128, 128, 3)  # Resize images to 128x128
batch_size = 64
num_classes = 26
epochs = 20

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

valid_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    "Augmented_data/train",
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode="categorical",
)

valid_generator = valid_datagen.flow_from_directory(
    "Augmented_data/valid",
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode="categorical",
)

# TCNN Model
def build_model_with_pretrained(input_shape, num_classes):
    base_model = tf.keras.applications.EfficientNetB0(
        include_top=False, input_shape=input_shape, weights="imagenet"
    )
    base_model.trainable = False  # Freeze pretrained weights
    
    model = Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(num_classes, activation="softmax"),
    ])
    return model

model = build_model_with_pretrained(input_shape, num_classes)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Learning Rate Scheduler and ReduceLROnPlateau
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=valid_generator.samples // batch_size,
    callbacks=[lr_reduction],
)

# Plot Loss and Accuracy
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

Found 19986 images belonging to 26 classes.
Found 7512 images belonging to 26 classes.


C:\Users\arshi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1115 - loss: 3.0347

C:\Users\arshi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


312/312 ━━━━━━━━━━━━━━━━━━━━ 1332s 4s/step - accuracy: 0.1115 - loss: 3.0346 - val_accuracy: 0.1333 - val_loss: 2.9155 - learning_rate: 0.0010
Epoch 2/20
  1/312 ━━━━━━━━━━━━━━━━━━━━ 7:59 2s/step - accuracy: 0.1250 - loss: 2.8751

C:\Users\arshi\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


312/312 ━━━━━━━━━━━━━━━━━━━━ 43s 133ms/step - accuracy: 0.1250 - loss: 2.8751 - val_accuracy: 0.0000e+00 - val_loss: 3.2395 - learning_rate: 0.0010
Epoch 3/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 1298s 4s/step - accuracy: 0.1240 - loss: 2.9538 - val_accuracy: 0.1327 - val_loss: 2.9169 - learning_rate: 0.0010
Epoch 4/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.1094 - loss: 2.8404 - val_accuracy: 0.1667 - val_loss: 3.0802 - learning_rate: 0.0010
Epoch 5/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 772s 2s/step - accuracy: 0.1337 - loss: 2.9383 - val_accuracy: 0.1321 - val_loss: 2.9161 - learning_rate: 5.0000e-04
Epoch 6/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 51s 161ms/step - accuracy: 0.0781 - loss: 2.9142 - val_accuracy: 0.3750 - val_loss: 2.6942 - learning_rate: 5.0000e-04
Epoch 7/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 676s 2s/step - accuracy: 0.1352 - loss: 2.9316 - val_accuracy: 0.1329 - val_loss: 2.9124 - learning_rate: 5.0000e-04
Epoch 8/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1094 - 

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Get the true labels and predictions
validation_steps = valid_generator.samples // valid_generator.batch_size + 1
y_true = valid_generator.classes  # True labels
y_pred_probs = model.predict(valid_generator, steps=validation_steps)  # Predicted probabilities
y_pred = np.argmax(y_pred_probs, axis=1)  # Predicted labels

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
class_names = list(valid_generator.class_indices.keys())  # Get class names

# Plot Confusion Matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# Classification Report
report = classification_report(y_true, y_pred, target_names=class_names)
print("Classification Report:\n")
print(report)
